## Model IO
모델 I/O 모듈에는 입력(input)과 출력(output)이 있음.
입력과 출력의 뜻으로 input은 우선 prompt(명령 등을 내리는 곳)임. (Prompts, Language models, Output parser 등이 있음.)

## Retrueval (리트리블)
- Retrueval은 `외부 데이터`로 접근하여 이를 모델에 `어떻게 제공`하는 것에 관한 거임.
- document loaders, Transformers, text embedding , vector stores , retrievers 등이 있음.
- 이것들 전부 우리의 데이터들로 작업하게 하고 모델에게 제공할 수 있는지에 관한 거임. ( 대충 외부 데이터를 가지고 모델과 작업하는 부분 )

## Chains
- 이전에 했던 내용. ( 나중에 자세히 작성 )

## Agents 
- `독립적으로 AI가 작동하도록 만들 수 있게 해주는 agents`.
- chains이 필요한 도구들을 선택하여 사용할 수 있도록 해줌.
- 그냥 chains에게 일을 맞기고 그에 맞는 커스텀 도구를 만들어 준다면 chains 스스로 사용할 tool들을 선택함...(?)

## Memory (기억)
- 챗봇에 Memory(기억)할 수 있도록 하는 것.

## Callbacks 
- callbacks는 기본적으로 model이 무엇을 하고 있는지, 중간에 알 수 있도록 하는 것. ( intermediate )
- 모델이 답변을 제공하기 전에 모델이 어떤 일을 하고 있는지 확인 할 수 있음. (무엇을 생각하고 있는지.)

In [1]:
# Template 처리.

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

template = PromptTemplate.from_template("What is the capital of {country}")

template.format(country="France")
# template.format() # <- error(필수 적인 값 France이 없기 떄문)

template = PromptTemplate(
    template="What is the capital of {country}",
    input_variables=["country"]
) # 이렇게도 가능.

template.format(country="France")

'What is the capital of France'

In [14]:
# FewShotPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)
# 미리 대답해 놓은 예시
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
    },
]

example_template = """
    Human: {question}
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)
# example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}") ## same

# 랭체인이 알아서 각각의 예제 리스트들을 prompt를 사용해서 형식화 할꺼임.
# question , answer 값으로 -> `이건 다르게 넣으면 안됨!`
# suffix는 형식화가 끝난 후 마지막에 나오는 거임.
# suffix에서 어떤 변수를 사용해야하는지 input_variables에 작성해 줘야함.
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

# prompt.format(country="Korea")
# 이렇게 해주면, 이제 어떻게 대답해야 하는지에 대한 형식을 알아서 형식에 맞게 대답해줌.! 대충 예제를 가져오고 예제를 prompt로 형식화 하고( 오브젝트 이름에 맞게 설정! example_template 참조.. ) 
# 그리고 형식화 된 값을 FewShot에게 전달해주기만 해도 양식에 맞춰서 대답해줌.. good..
chain = prompt | chat

chain.invoke({
    "country" : "Korea"
})

AI: 
I know this:
Capital: Seoul
Language: Korean
Food: Kimchi and Bibimbap
Currency: South Korean Won

AIMessageChunk(content='AI: \nI know this:\nCapital: Seoul\nLanguage: Korean\nFood: Kimchi and Bibimbap\nCurrency: South Korean Won')

## FewShotPromptTemplate , Fewshot Learning
- `Fewshot은 모델에게 예제를 준다는 뜻과 같음.`
- 더 나은 대답을 할 수 있도록 하는 예제를 줄 수 있음.
- 복잡한 대답을 원할 경우 모델에게 `어떻게 대답해야 하는 지에 대한 예제를 AI모델에게 주는게(Fewshot) prompt를 사용해서 어떻게 대답해야 하는지 알려주는 것보다 훨씬 좋음.`
- 예를 들어 모델에게 콤마(,)를 써서 구분해줘, 소문자만 써야해 등... ( 모델이 text를 생성하기에 어떻게 대답해야 하는지에 대한 예시를 미리 주는게 더 좋은 방식임! `생성하기 전에 예시를 줌!` 생성 할 경우에 주는게 아니라! )
- 만약 고객지원 봇을 만들때 이전 데이터( 이전에 운영 했을 경우 )가 있을 경우 많은 고객들과 대화 기록이 남아 있을 것임. 그럼 language model에게 어떻게 말해야 할지(대응 해야할지)에 대해 알려주고 싶을꺼임.(많은 데이터가 있기 때문에) 그럼 그냥 대화 내용을 가져와서 형식화 시켜주면 더 잘 만들 수 있을것임.
- `FewShotPromptTemplate`은 형식을 주고 그 형식대로 만들어 돌라는 것을 하는거임! 엄청난 기능!

In [20]:
# FewShotChatMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)
# 미리 대답해 놓은 예시
examples = [
    {
        "country": "France",
        "answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
    },
    {
        "country": "Italy",
        "answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
    },
    {
        "country": "Greece",
        "answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
    },
]


# 다른 부분.
# AI에게 "너 이런식으로 답변했었어." 
# 으로 답변하게 만들려고 AI를 속이고 있는 방식.
# 예제를 형식화 하기 위한것. -> 실제 메세지 x 오직 형식화. 밑에서 형식으로 변환!
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {country}?"),
    ("ai", "{answer}")
])
# example_prompt(이전에 이렇게 답했어.)대답 양식을 이용한 fewshot생성
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
# 이전에 답했던?(example_prompt)를 이용해서 chat처럼 답하게 작성.
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "you are a geography expert. you give short answers."),
    example_prompt,
    ("human", "What do you know about {country}?"),
])


chain = final_prompt | chat

chain.invoke({
    "country": "South Korea"
})


I know this:
Capital: Seoul
Language: Korean
Food: Kimchi and Bibimbap
Currency: South Korean Won

AIMessageChunk(content='\nI know this:\nCapital: Seoul\nLanguage: Korean\nFood: Kimchi and Bibimbap\nCurrency: South Korean Won')

In [42]:
# Length Based Example Selector
# sample이 많아지는 경우가 있어, 어느 정도의 example들을 골라서 prompt에 허용할건지 정해야하는 경우에 사용.

from typing import Any, Dict, List
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

# 미리 대답해 놓은 예시
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# 예제 선택기를 만드는 방법!. BaseExampleSelector를 가져와서 만들면됨. 
# 부족한 부분은 이거 만들어야 된다 error 출력됨.. ! 굿..
class RandomExampleSelector(BaseExampleSelector):

    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]


example_prompt = PromptTemplate.from_template(
    "Human: {question}\nAI: {answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

""" example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=80
)
 """
# print(example_selector.example_text_lengths)
# [73, 71, 72] 가 나오는데 80 제한이라 하나만 가져와 지는것.! 예제들의 크기 가 합산임!
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

print(prompt.format(country="south korea"))

Human: What do you know about Italy?
AI: 
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        

Human: What do you know about south korea?


' chain = prompt | chat\n\nchain.invoke({\n    "country": "Korea"\n}) '

In [6]:
# Serialization and Composition
# prompt를 어디 저장하고 다른 누구나 플롬포트를 가져다 쓸 수 있도록 하고 싶은 경우.
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt
from langchain.prompts.pipeline import PipelinePromptTemplate

# json
# prompt = load_prompt("../prompt.json")

# yaml
# prompt = load_prompt("../prompt.yaml")
# prompt.format(country="south korea")

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start)
]
# 여러개의 prompt들을 하나의 prompt로 합치는 방법.
full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)

# format test임! 
full_prompt.format(
    character="Pirate",
    example_question="What is your location?",
    example_answer="Arrrrrrg! That is a secret!!! Arrrrg! Arrrrg!",
    question="What is your fav food?",
)

chain = full_prompt | chat
chain.invoke({
    "character": "Pirate",
    "example_question": "What is your location?",
    "example_answer": "Arrrrrrg! That is a secret!!! Arrrrg! Arrrrg!",
    "question": "What is your fav food?",
})

Arrrrg! Me favorite food be a hearty stew made with fresh seafood and plenty of spices! Arrrrg! Nothing like a good meal to keep me strength up for plunderin' the high seas! Arrrrg!

AIMessageChunk(content="Arrrrg! Me favorite food be a hearty stew made with fresh seafood and plenty of spices! Arrrrg! Nothing like a good meal to keep me strength up for plunderin' the high seas! Arrrrg!")